<a href="https://colab.research.google.com/github/greek-proverb-atlas/proverbs.gr/blob/main/tuning_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tuning ElasticNet
* The best traditional ML regressor
* Repeating the experiments after correcting the coordinates for specific locations


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
!pip install optuna
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

import ast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.7 MB/s eta 0:00:00


In [2]:
root = 'https://github.com/greek-proverb-atlas/proverbs.gr/raw/refs/heads/main'
balanced_corpus = pd.read_csv(f"{root}/data/balanced_corpus.csv", index_col=0)
# quality assurance patch
geolocs = pd.read_csv(f'{root}/data/geolocs.csv')
balanced_corpus.lat = balanced_corpus.area.apply(lambda x: geolocs[x].iloc[0])
balanced_corpus.lon = balanced_corpus.area.apply(lambda x: geolocs[x].iloc[1])

In [9]:
train, test = train_test_split(balanced_corpus, test_size=0.05, random_state=2023)
train, dev = train_test_split(train, test_size=test.shape[0], random_state=2023)

def objective(trial):

    alpha = trial.suggest_loguniform('alpha', 1e-4, 1.0)
    l1_ratio = trial.suggest_uniform('l1_ratio', 0, 1)
    regressor = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

    vect = TfidfVectorizer(ngram_range=(2,5), analyzer="char", max_df=0.5,
                           min_df=10, lowercase=True)

    pipe = Pipeline([('vect', vect), ('ref', regressor)])
    scores = cross_val_score(pipe,
                             train.text.values,
                             train[["lat", "lon"]].values,
                             cv=5,
                             n_jobs=-1)
    return scores.mean()


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-11-07 10:34:59,730] A new study created in memory with name: no-name-6ebda72e-cc70-4db2-97bc-7f5a522277c2
<ipython-input-9-53914a76af18>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-4, 1.0)
<ipython-input-9-53914a76af18>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  l1_ratio = trial.suggest_uniform('l1_ratio', 0, 1)
[I 2024-11-07 10:35:03,878] Trial 0 finished with value: 0.08255861565967484 and parameters: {'alpha': 0.0032400847808073275, 'l1_ratio': 0.26934676936472246}. Best is trial 0 with value: 0.08255861565967484.
<ipython-input-9-53914a76af18>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

In [15]:
print('best value:', study.best_value)
print('best trial:', study.best_trial)
best_params = study.best_trial.params
print('best params:', best_params)

best value: 0.26887607129732816
best trial: FrozenTrial(number=92, state=TrialState.COMPLETE, values=[0.26887607129732816], datetime_start=datetime.datetime(2024, 11, 7, 11, 13, 3, 914367), datetime_complete=datetime.datetime(2024, 11, 7, 11, 13, 36, 345434), params={'alpha': 0.00019221567526237228, 'l1_ratio': 0.4300483833385914}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'alpha': FloatDistribution(high=1.0, log=True, low=0.0001, step=None), 'l1_ratio': FloatDistribution(high=1.0, log=False, low=0.0, step=None)}, trial_id=92, value=None)
best params: {'alpha': 0.00019221567526237228, 'l1_ratio': 0.4300483833385914}


In [11]:
regressor = ElasticNet(**best_params)
vect = TfidfVectorizer(ngram_range=(2,5), analyzer="char", max_df=0.5, min_df=10, lowercase=True)
elastic_net_pipe = Pipeline([('vect', vect), ('reg', regressor)])
elastic_net_pipe.fit(train.text.values, train[["lat", "lon"]].values)
score = elastic_net_pipe.score(test.text.values, test[["lat", "lon"]].values)
score

0.29666617596641737

In [12]:
predicted_coords = elastic_net_pipe.predict(test.text.values)

mae_lat, mae_lon = (mean_absolute_error(test.lat.values, predicted_coords[:, 0]),
                    mean_absolute_error(test.lon.values, predicted_coords[:, 1]))
mse_lat, mse_lon = (mean_squared_error(test.lat.values, predicted_coords[:, 0]),
                    mean_squared_error(test.lon.values, predicted_coords[:, 1]))

print(mae_lat, mae_lon)
print(mse_lat, mse_lon)

1.2984441549383554 2.4986161394128756
2.6321071350509873 12.039591580667107


In [13]:
reg_models = {'el':[]}
scores = {m:{'mse':{'lat':[], 'lon':[]}, 'mae':{'lat':[], 'lon':[]}} for m in reg_models}
for i in range(2,6):
    balanced_corpus = pd.read_csv(f"{root}/data/balanced_corpus_{i}.csv", index_col=0).reset_index()
    # quality assurance patch
    geolocs = pd.read_csv(f'{root}/data/geolocs.csv')
    balanced_corpus.lat = balanced_corpus.area.apply(lambda x: geolocs[x].iloc[0])
    balanced_corpus.lon = balanced_corpus.area.apply(lambda x: geolocs[x].iloc[1])
    # end of quality assurance
    train, test = train_test_split(balanced_corpus, test_size=0.05, random_state=2023)
    train, dev = train_test_split(train, test_size=test.shape[0], random_state=2023)

    vect = TfidfVectorizer(ngram_range=(2,5), analyzer="char", max_df=0.5,
                           min_df=10, lowercase=True)
    # elastic
    el_reg = Pipeline([('vect', vect), ('reg', ElasticNet(**best_params))])
    el_reg.fit(train.text.values, train[["lat", "lon"]].values)
    reg_models['el'] = el_reg

In [18]:
for i in range(2,6):

  for model_name in reg_models:
          regressor = reg_models[model_name]
          preds = regressor.predict(test.text.values)
          mae_lat, mae_lon = (mean_absolute_error(test.lat.values, preds[:, 0]),
                              mean_absolute_error(test.lon.values, preds[:, 1]))
          mse_lat, mse_lon = (mean_squared_error(test.lat.values, preds[:, 0]),
                              mean_squared_error(test.lon.values, preds[:, 1]))
          scores[model_name]['mae']['lat'].append(mae_lat)
          scores[model_name]['mae']['lon'].append(mae_lon)
          scores[model_name]['mse']['lat'].append(mse_lat)
          scores[model_name]['mse']['lon'].append(mse_lon)

In [19]:
pd.DataFrame({'mae-lon':pd.DataFrame({m:scores[m]['mae']['lon'] for m in reg_models}).mean(),
              'mae-lat':pd.DataFrame({m:scores[m]['mae']['lat'] for m in reg_models}).mean(),
              'mse-lon':pd.DataFrame({m:scores[m]['mse']['lon'] for m in reg_models}).mean(),
              'mse-lat':pd.DataFrame({m:scores[m]['mse']['lat'] for m in reg_models}).mean()})

,mae-lon,mae-lat,mse-lon,mse-lat
el,2.683873,1.272525,12.990383,2.561877


# GrBERT results

In [22]:
data = {
#    "Run": ["Run 1", "Run 2", "Run 3"],
    "MAE_lon": [1.61, 1.67, 1.80],
    "MAE_lat": [1.22, 1.30, 1.22],
    "MSE_lon": [4.32, 4.58, 5.58],
    "MSE_lat": [2.44, 2.68, 2.41],
    "R2_lat": [-1.03, -1.79, -1.19],
    "R2_lon": [-0.39, -0.70, -0.40]
}
pd.DataFrame(data).mean(0)

,0
MAE_lon,1.693333
MAE_lat,1.246667
MSE_lon,4.826667
MSE_lat,2.510000
R2_lat,-1.336667
R2_lon,-0.496667
